In [0]:
%%cu
#include <stdio.h>
#include <stdlib.h>

/*
This program computes and displays the transpose of an N × N matrix entered by the user.
*/

__global__ void matrixTranspose (float * d_matrix, float * d_transpose, int N)
{
    int row = blockIdx.y * N + threadIdx.y;
    int column = blockIdx.x * N + threadIdx.x;

    if (row < N && column < N)
    {
        for (int i = 0; i < N; i++)
            d_transpose[row * N + i] = d_matrix[i * N + column];
    }
}

int main()
{
    int N=3;

    //declaring input, output matrices
    float h_matrix[30] = {1,2,3,4,5,6,7,8,9};
    float h_transpose[30] = {0};

    //asking user for size of square matrix
    printf("Enter the no. of rows in square matrix. \n");
    scanf ("%d", &N);

    int totalElements = N * N;
    int matrixBytes = totalElements * sizeof(float);


    //getting user input for elements of matrix
    printf("Enter the elements in square matrix. \n");
    for (int i = 0; i < totalElements; i++)
        scanf("%f", &h_matrix[i]);

    //display input matrix
    printf("The matrix entered is:");

    for (int i = 0; i < N; i++)
    {
        printf("\n");
        for (int j = 0; j < N; j++)
            printf("%0.1f \t", h_matrix[i * N + j]);
    }

    //declare GPU memory pointers
    float * d_matrix;
    float * d_transpose;

    //allocate memory on GPU
    cudaMalloc ((void**) &d_matrix, matrixBytes);
    cudaMalloc ((void**) &d_transpose, matrixBytes);

    //transfer input array to GPU
    cudaMemcpy (d_matrix, h_matrix, matrixBytes, cudaMemcpyHostToDevice);

    //launch kernel
    matrixTranspose<<<1 , dim3(N,N)>>>(d_matrix, d_transpose, N);

    //copy output array back to host
    cudaMemcpy (h_transpose, d_transpose, matrixBytes, cudaMemcpyDeviceToHost);

    //display output array
    printf("\nThe transposed matrix is:\n");

    for (int i = 0; i < N; i++)
    {
        printf ("\n");
        for (int j = 0; j < N; j++)
            printf("%0.1f \t", h_transpose[i * N + j]);
    }

    return 0;
}
